In [ ]:
import numpy as np
import psdr, psdr.demos
from scipy.linalg import orth
from psdr import local_linear_grads

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Build a test function
np.random.seed(0)
m = 5
Q = orth(np.random.randn(m,m))
lam = np.zeros(m)
lam = np.linspace(1,10, m)**(-2)
A = Q @ np.diag(lam) @ Q.T
fun = psdr.demos.QuadraticFunction(quad = A)


# Estimating Gradients from Samples Using a Perplexity-based Bandwidth

In many cases we seek the gradient of a function $\nabla f(\mathbf{x})$, but only have access to samples $f(\mathbf{x}_i)$ where the sample locations $\lbrace\mathbf{x}_i\rbrace_{i=1}^N$ are pre-determined and may be arbitrary; i.e., we cannot employ the finite difference approaches that are natural if we are free to choose $\mathbf{x}_i$.  In this situation, we will seek to estimate the gradient at $\mathbf{x}_i$ using local linear models centered around $\mathbf{x}_i$ motivated by its use in the "Outer Product Gradient" of Hardle and Stoker (1989).  This same approach appears in earlier texts; see, e.g., "Local Polynomial Modelling and Its Applications" by Jianqing Fan and Irene Gijbels Sec. 2.3.

The basic premise is as follows.  Suppose we have a kernel function, say $k_\beta(\mathbf{x}_1, \mathbf{x}_2) = e^{-\beta \|\mathbf{x}_1 - \mathbf{x}_2\|_2^2}$; we then solve a weighted linear system to construct the local linear model $g(\mathbf{x}) = a_0 + \mathbf{a}^\top \mathbf{x}$
around $\mathbf{x}_j$:
$$ 
\min_{a_0\in \mathbb{R}, \mathbf{a}\in \mathbb{R}^m}
\sum_{i=1}^N [(a_0 + \mathbf{a}^\top \mathbf{x}_i) - f(\mathbf{x}_i)]^2 k_\beta(\mathbf{x}_i, \mathbf{x}_j).
$$
## The Importance of Bandwidth
The choice of the kernel and its bandwidth $\beta$ play a critical role in the efficacy of this approach. We do not want the bandwidth to be too small as then all the weights will be small yielding an ill-conditioned system; nor do we want the bandwidth to be too large as then the weights are all effectively one and the resulting global linear model will ignore the nonlinearlities of $f$.  The figure below shows the effect of this Goldilocks effect.

In [ ]:
X = fun.domain.sample(1000)
fX = fun(X)
grads_true = fun.grad(X)

fig, ax = plt.subplots()
xx = np.linspace(0,1, 100)
for i, bw in enumerate([2, 20, 200]):
    grads = local_linear_grads(X, fX, bandwidth = bw)
    err = np.max(np.abs(grads - grads_true), axis = 1)
    ax.hist(err, xx, label = 'bw=%g' % bw, alpha = 0.5)
ax.legend();

There are a variety of approaches for computing bandwidth including rule of thumb estimates and cross-validation.  Here we argue for an approach based on Perplexity.

## Perplexitity Based Bandwidth
In algorithms like the SNE and t-SNE, an important step in processing the original high dimensional similarity between two points is to pick a bandwidth at each sample such that the perplexity---the entropy associated with the neigboring points---has a particular value. This ensures that areas of high and low sample density are more fairly treated.  Our approach here is to compute a per-sample bandwidth such that the perplexity is $m+1$.  Although this increases computation time compared to the fixed bandwidth approach with a value taken from Xia 2007 (see Li 18, eq. 11.5), it yields fare more accurate gradient estimates.


### Example: Random Data
With purely random data, we should expect that with the optimum bandwidth should be roughly the same for each sample point. Here we see that if we avoid the heuristic proposed by Xia, we can obtain answers comparable to the perplexity based approach.

In [ ]:
X = fun.domain.sample(2000)
fX = fun(X)
grads_true = fun.grad(X)

print("OPG")
%time grads1 = local_linear_grads(X, fX, bandwidth = 'xia')
grads2 = local_linear_grads(X, fX, bandwidth = 20)
print("Perplexity OPG")
%time grads3 = local_linear_grads(X, fX, perplexity = m+1)
err1 = np.max(np.abs(grads1 - grads_true), axis = 1)
err2 = np.max(np.abs(grads2 - grads_true), axis = 1)
err3 = np.max(np.abs(grads3 - grads_true), axis = 1)

fig, axes = plt.subplots(1,2, figsize = (14,7))
ax = axes[0]
xx = np.linspace(0, max(np.max(err1), np.max(err2)), 50)
ax.hist(err1, xx, alpha = 0.5, label = 'opg')
ax.hist(err2, xx, alpha = 0.5, label = 'opg opt bw');
ax.hist(err3, xx, alpha = 0.5, label = 'popg');
ax.set_xlabel('error, inf-norm')
ax.set_ylabel('density')
ax.legend();

ax = axes[1]
ax.plot(np.sort(err1), label = 'opg');
ax.plot(np.sort(err2), label = 'opg opt bw')
ax.plot(np.sort(err3), label = 'popg')
ax.set_yscale('log')
ax.legend()
ax.set_xlabel('index')
ax.set_ylabel('error inf-norm');

### Example: Mixed Data
In this example, data is drawn from two distributions to reflect the challenges inherent when we have data that doesn't emerge from a simple random sample approach.  Here, the static bandwidth approach breaks down.

In [ ]:
X1 = fun.domain.sample(1000)
# We place this off-center so that gradients are large there
X2 = 0.1*fun.domain.sample(1000) + 0.5*np.ones(len(fun.domain))
X = np.vstack([X1,X2])
fX = fun(X)
grads_true = fun.grad(X)

print("OPG")
%time grads1 = local_linear_grads(X, fX, bandwidth = 'xia')
grads2 = local_linear_grads(X, fX, bandwidth = 20)
print("Perplexity OPG")
%time grads3 = local_linear_grads(X, fX, perplexity = m+1)
err1 = np.max(np.abs(grads1 - grads_true), axis = 1)
err2 = np.max(np.abs(grads2 - grads_true), axis = 1)
err3 = np.max(np.abs(grads3 - grads_true), axis = 1)

fig, axes = plt.subplots(1,2, figsize =(14,7))
ax = axes[0]
xx = np.linspace(0, max(np.max(err1), np.max(err2)), 200)
ax.hist(err1, xx, alpha = 0.5, label = 'opg')
ax.hist(err2, xx, alpha = 0.5, label = 'opg opt bw');
ax.hist(err3, xx, alpha = 0.5, label = 'popg');
ax.set_xlabel('error, inf-norm')
ax.set_ylabel('density')
ax.legend();

ax = axes[1]
ax.plot(np.sort(err1), label = 'opg');
ax.plot(np.sort(err2), label = 'opg opt bw')
ax.plot(np.sort(err3), label = 'popg')
ax.set_yscale('log')
ax.legend()
ax.set_xlabel('index')
ax.set_ylabel('error inf-norm');